In [1]:
class AHPNode:
    def __init__(self, name, level):
        self.name = name
        self.level = level
        self.children = []
        self.parent = None

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

    def __repr__(self):
        return f"{self.level}: {self.name}"

In [3]:
# Create the goal
goal = AHPNode("Select Best Waste-to-Energy Technology", "Goal")

# Main Criteria
technological = AHPNode("Technological", "Main Criterion")
economical = AHPNode("Economical", "Main Criterion")
environmental = AHPNode("Environmental", "Main Criterion")
socio_cultural = AHPNode("Socio-cultural", "Main Criterion")

goal.add_child(technological)
goal.add_child(economical)
goal.add_child(environmental)
goal.add_child(socio_cultural)

# --- TECHNOLOGICAL SUBSTRUCTURE ---
feedstock = AHPNode("Feedstock", "Subcriterion")
energy_output = AHPNode("Energy Output", "Subcriterion")
efficiencies = AHPNode("Efficiencies", "Subcriterion")
scalability = AHPNode("Scalability", "Subcriterion")

technological.add_child(feedstock)
technological.add_child(energy_output)
technological.add_child(efficiencies)
technological.add_child(scalability)

feedstock.add_child(AHPNode("Moisture Content (%)", "Indicator"))
feedstock.add_child(AHPNode("Calorific Value (Mg/kg)", "Indicator"))
feedstock.add_child(AHPNode("Pre-Processing", "Indicator"))

scalability_complexity = AHPNode("Technology Complexity", "Deciding Factor")
scalability_maturity = AHPNode("Technology Maturity", "Deciding Factor")
scalability.add_child(scalability_complexity)
scalability.add_child(scalability_maturity)

scalability_complexity.add_child(AHPNode("Retrofitting Feasibility", "Indicator"))
scalability_complexity.add_child(AHPNode("No. of processing steps", "Indicator"))

scalability_maturity.add_child(AHPNode("Technological Readiness", "Indicator"))
scalability_maturity.add_child(AHPNode("No. of commercial-scale plants", "Indicator"))

energy_output.add_child(AHPNode("Energy Output Quality", "Indicator"))
efficiencies.add_child(AHPNode("Conversion Efficiency", "Indicator"))

# --- ECONOMICAL SUBSTRUCTURE ---
capex = AHPNode("CAPEX (EUR/Ton)", "Subcriterion")
opex = AHPNode("OPEX (EUR/Ton)", "Subcriterion")
lcoe = AHPNode("LCOE (EUR/MJ)", "Subcriterion")

economical.add_child(capex)
economical.add_child(opex)
economical.add_child(lcoe)

capex.add_child(AHPNode("Capital Cost Indicator", "Indicator"))
opex.add_child(AHPNode("Operating Cost Indicator", "Indicator"))
lcoe.add_child(AHPNode("Levelized Cost Indicator", "Indicator"))

# --- ENVIRONMENTAL SUBSTRUCTURE ---
waste_reduction = AHPNode("Waste Volume Reduction Potential (%)", "Subcriterion")
air_pollutants = AHPNode("Air Pollutants", "Subcriterion")
byproduct = AHPNode("Byproduct", "Subcriterion")
water_pollutants = AHPNode("Water Pollutants", "Subcriterion")

environmental.add_child(waste_reduction)
environmental.add_child(air_pollutants)
environmental.add_child(byproduct)
environmental.add_child(water_pollutants)

waste_reduction.add_child(AHPNode("Waste Reduction Efficiency", "Indicator"))

air_pollutants.add_child(AHPNode("CO2 (kg eq CO2/ton)", "Indicator"))
air_pollutants.add_child(AHPNode("Nox (kg eq CO2/ton)", "Indicator"))
air_pollutants.add_child(AHPNode("PM", "Indicator"))
air_pollutants.add_child(AHPNode("SO2 (g/t)", "Indicator"))

byproduct.add_child(AHPNode("Ash/Char Residues (kg/ton MSW)", "Indicator"))
byproduct.add_child(AHPNode("Trace Metals (mg/ton)", "Indicator"))

water_pollutants.add_child(AHPNode("Organic Compounds", "Indicator"))
water_pollutants.add_child(AHPNode("Heavy Metals", "Indicator"))
water_pollutants.add_child(AHPNode("Neutrants (N,P)", "Indicator"))
water_pollutants.add_child(AHPNode("Suspended Solids", "Indicator"))

# --- SOCIO-CULTURAL SUBSTRUCTURE ---
socio_cultural.add_child(AHPNode("Employment / Automation Focused", "Subcriterion"))
socio_cultural.add_child(AHPNode("Worker Exposure Risk", "Subcriterion"))
socio_cultural.add_child(AHPNode("NIMBY Effect", "Subcriterion"))

In [4]:
alternatives = [
    "Composting", "AD", "HTC", "PYR", "GAS",
    "INC-H", "INC-E", "INC-H-85", "INC-E-85", "INC-H-95", "INC-E-95"
]

alt_nodes = [AHPNode(name, "Alternative") for name in alternatives]

def attach_alternatives_to_leaves(node):
    if not node.children and node.level != "Alternative":
        for alt in alt_nodes:
            node.add_child(AHPNode(alt.name, "Alternative"))
    else:
        for child in node.children:
            attach_alternatives_to_leaves(child)

attach_alternatives_to_leaves(goal)

In [5]:
def print_hierarchy(node, level=0):
    print("    " * level + f"- {node.name} ({node.level})")
    for child in node.children:
        print_hierarchy(child, level + 1)

print_hierarchy(goal)

- Select Best Waste-to-Energy Technology (Goal)
    - Technological (Main Criterion)
        - Feedstock (Subcriterion)
            - Moisture Content (%) (Indicator)
                - Composting (Alternative)
                - AD (Alternative)
                - HTC (Alternative)
                - PYR (Alternative)
                - GAS (Alternative)
                - INC-H (Alternative)
                - INC-E (Alternative)
                - INC-H-85 (Alternative)
                - INC-E-85 (Alternative)
                - INC-H-95 (Alternative)
                - INC-E-95 (Alternative)
            - Calorific Value (Mg/kg) (Indicator)
                - Composting (Alternative)
                - AD (Alternative)
                - HTC (Alternative)
                - PYR (Alternative)
                - GAS (Alternative)
                - INC-H (Alternative)
                - INC-E (Alternative)
                - INC-H-85 (Alternative)
                - INC-E-85 (Alternative)
             

In [10]:
import numpy as np

def compute_ahp_weights_from_raw_data(raw_values: dict, reverse: bool = True):
    """
    Given a dictionary of raw data (e.g., CAPEX values),
    builds Saaty matrix, calculates weights, and checks consistency.
    
    reverse = True means lower values are better (cost, emissions, etc.)
    """
    names = list(raw_values.keys())
    size = len(names)
    matrix = np.ones((size, size))

    # Fill matrix
    for i in range(size):
        for j in range(size):
            if i == j:
                continue
            a = raw_values[names[i]]
            b = raw_values[names[j]]
            ratio = (b / a) if reverse else (a / b)
            matrix[i][j] = min(max(ratio, 1/9), 9)

    # Normalize and get priority vector
    norm_matrix = matrix / matrix.sum(axis=0)
    priority_vector = norm_matrix.mean(axis=1)

    # Consistency check
    weighted_sum = np.dot(matrix, priority_vector)
    lambda_max = (weighted_sum / priority_vector).mean()
    ci = (lambda_max - size) / (size - 1)
    ri_values = {1: 0.0, 2: 0.0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32,
                 8: 1.41, 9: 1.45, 10: 1.49, 11: 1.51}
    ri = ri_values.get(size, 1.51)
    cr = ci / ri if ri else 0

    results = {
        "priority_vector": dict(zip(names, priority_vector)),
        "lambda_max": lambda_max,
        "ci": ci,
        "cr": cr,
        "matrix": matrix
    }
    return results

In [11]:
capex_midpoints = {
    "INC-H": 428.0,
    "INC-E": 591.0,
    "AD": 245.0,
    "PYR": 492.0,
    "GAS": 604.5,
    "HTC": 314.0,
    "Composting": 139.0,
    "INC-H-85": 430.0,
    "INC-H-95": 433.5,
    "INC-E-85": 596.5,
    "INC-E-95": 597.5
}

capex_result = compute_ahp_weights_from_raw_data(capex_midpoints, reverse=True)

for name, weight in capex_result["priority_vector"].items():
    print(f"{name}: {weight:.4f}")

print("\nλ_max:", round(capex_result["lambda_max"], 4))
print("CI:", round(capex_result["ci"], 5))
print("CR:", round(capex_result["cr"], 5))

INC-H: 0.0775
INC-E: 0.0561
AD: 0.1353
PYR: 0.0674
GAS: 0.0549
HTC: 0.1056
Composting: 0.2385
INC-H-85: 0.0771
INC-H-95: 0.0765
INC-E-85: 0.0556
INC-E-95: 0.0555

λ_max: 11.0
CI: 0.0
CR: 0.0
